In [ ]:
import os
import glob
import numpy as np
import MDAnalysis as mda
import MDAnalysis.analysis.rms
from os.path import basename
import MDAnalysis.analysis.rms

from matplotlib import pyplot as plt

In [ ]:
REF_ATOMISTIC = "backmapped/A2A/atomistic/protein_from_PED_model/true_0.pdb"
OUR_BACKMAPPING = "backmapped/A2A/atomistic/protein_from_PED_model/minimized/backmapped_0.pdb"
THEIR_BACKMAPPING = "backmapped/A2A/CG/protein_from_cg2at/de_novo_min_31.pdb"

PROTEIN_NO_CAPS = "protein and not (resname ACE NME)"
SELECT_ALL_SIDE_CHAINS = 'protein and not (resname ACE NME) and not (type H or type OW) and resid 0 to 290'
SELECT_BB = 'protein and not (resname ACE NME) and not (type H or type OW) and resid 0 to 290 and name CA C N O'
SELECT = SELECT_ALL_SIDE_CHAINS

u_ref = mda.Universe(REF_ATOMISTIC)
protein_ref = u_ref.select_atoms(PROTEIN_NO_CAPS)
protein_ref_min_resnum =  protein_ref.residues[0].resid
for r in protein_ref.residues:
    r.resid -= protein_ref_min_resnum

u_our = mda.Universe(OUR_BACKMAPPING)
protein_our = u_our.select_atoms(PROTEIN_NO_CAPS)
protein_our_min_resnum = protein_our.residues[0].resid
for r in protein_our.residues:
    r.resid -= protein_our_min_resnum

u_their = mda.Universe(THEIR_BACKMAPPING)
protein_their = u_their.select_atoms(PROTEIN_NO_CAPS)
protein_their_min_resnum = protein_their.residues[0].resid
for r in protein_their.residues:
    r.resid -= protein_their_min_resnum

In [ ]:
protein_ref.write(REF_ATOMISTIC.replace(".pdb", "_nocap.pdb"))
protein_our.write(OUR_BACKMAPPING.replace(".pdb", "_nocap.pdb"))
# protein_their.write(THEIR_BACKMAPPING)

In [ ]:
R_our = MDAnalysis.analysis.rms.RMSD(u_ref, u_our,
           select=SELECT_BB,
           groupselections=[
               SELECT_ALL_SIDE_CHAINS
            ])
R_our.run()
R_their = MDAnalysis.analysis.rms.RMSD(u_ref, u_their,
           select=SELECT_BB,
           groupselections=[
               SELECT_ALL_SIDE_CHAINS
            ])
R_their.run()

rmsd_our = R_our.rmsd.T
rmsd_their = R_their.rmsd.T
time = rmsd_our[1]*10
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.plot(time, rmsd_our[2], 'b-',  label="BB")
ax.plot(time, rmsd_our[3], 'b--', label="ALL")
ax.plot(time, rmsd_their[2], 'k-',  label="BB")
ax.plot(time, rmsd_their[3], 'k--', label="ALL")
ax.legend(loc="best")
ax.set_xlabel("Frame")
ax.set_ylabel(r"RMSD ($\AA$)")

In [ ]:
ROOT_TARGET = "backmapped/PED/atomistic/PED00055/"
TAG_TARGET = "backmapped"
ROOT_REF = "backmapped/PED/atomistic/PED00055"
TAG_REF = "true"

rmsd = []
for trg_filename in glob.glob(os.path.join(ROOT_TARGET, f"{TAG_TARGET}*.pdb")):
    src_filename = os.path.join(ROOT_REF, basename(trg_filename).replace(TAG_TARGET, TAG_REF))
    u = MDAnalysis.Universe(trg_filename)
    ref = MDAnalysis.Universe(src_filename)

    R = MDAnalysis.analysis.rms.RMSD(u, ref,
            select="protein and not (name H* or type OW)",             # superimpose on whole backbone of the whole protein
            groupselections=[
                "backbone and not (name H* or type OW)",
                "protein and (not backbone) and not (name H* or type OW)"
            ])
    R.run()
    frame_rmsd = R.rmsd[:, 2:]
    rmsd.append(frame_rmsd)
rmsd = np.concatenate(rmsd, axis=0).T

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.hist(rmsd[0], density=True, bins=20, histtype='step', linewidth=1, edgecolor='red',fill=False, label="All Heavy Atoms")
# ax.hist(rmsd[1], density=False, bins=20, histtype='step', linewidth=1, edgecolor='orange',fill=False, label="BB")
# ax.hist(rmsd[2], density=False, bins=20, histtype='step', linewidth=1, edgecolor='c',fill=False, label="Side-Chains")

all, BB, SC = rmsd[:, np.argmax(rmsd[0])]
ax.vlines(all, 0, 18, colors='k', linestyles='--', label='')
# ax.vlines(BB, 0, 8, colors='orange', linestyles='--', label='')
# ax.vlines(SC, 0, 8, colors='c', linestyles='--', label='')

from scipy.stats import norm
mu, std = norm.fit(rmsd[0])

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)

ax.plot(x, p, 'k', linewidth=1)

ax.legend(loc="best")
ax.set_xlabel(r"RMSD ($\AA$)")
ax.set_ylabel("# of Frames")
plt.title("RMSD Distribution for PED00055 - Minimized Structures")
# fig.savefig("rmsd_all_CORE_LID_NMP_ref1AKE.pdf")

In [ ]:
import openmm

In [ ]:
import xml.etree.ElementTree as etree
from copy import deepcopy

# Define file paths

charmm_ffxml_filename = '../charmm/ffxml/charmm36_nowaters.xml' # input CHARMM ffxml file containing lipid definitions
amber_unmerged_lipid_ffxml_filename = 'ffxml/lipid17.xml' # input AMBER ffxml file for unmerged lipids
charmmlipid2amber_filename = 'files/charmmlipid2amber.csv' # AMBER to CHARMM lipid residue conversion table
amber_merged_lipid_ffxml_filename = 'ffxml/lipid17_merged.xml' # output AMBER ffxml file for merged lipids

# Read the input files.

charmmff = etree.parse(charmm_ffxml_filename)
amberff = etree.parse(amber_unmerged_lipid_ffxml_filename)
charmmResidues = charmmff.getroot().find('Residues').findall('Residue')
amberResidues = amberff.getroot().find('Residues').findall('Residue')
amberResMap = {}
for res in amberResidues:
    atoms = dict((atom.attrib['name'], atom) for atom in res.findall('Atom'))
    amberResMap[res.attrib['name']] = atoms
translations = {}
with open(charmmlipid2amber_filename) as input:
    # Skip the first two lines.
    input.readline()
    input.readline()
    for line in input:
        fields = line.split(',')
        mergedRes = fields[0]
        mergedAtom = fields[2].split()[0]
        originalAtom, originalRes = fields[3].split()
        translations[(mergedRes, mergedAtom)] = (originalRes, originalAtom)

# Remove all residues from the Amber file.

parentNode = amberff.getroot().find('Residues')
for res in amberResidues:
    parentNode.remove(res)

# Copy over the CHARMM residues, making appropriate replacements.

def translateResidue(residue):
    newres = deepcopy(residue)
    for atom in newres.findall('Atom'):
        key = (residue.attrib['name'], atom.attrib['name'])
        if key not in translations:
            return None # We don't have a translation.
        amberResName, amberAtomName = translations[key]
        if amberResName not in amberResMap or amberAtomName not in amberResMap[amberResName]:
            return None # We don't have a translation.
        amberAtom = amberResMap[amberResName][amberAtomName]
        for attrib in amberAtom.attrib:
            if attrib != 'name':
                atom.attrib[attrib] = amberAtom.attrib[attrib]
    return newres

for residue in charmmResidues:
    copy = translateResidue(residue)
    if copy is not None:
        parentNode.append(copy)

# Write merged lipid ffxml file

amberff.write(amber_merged_lipid_ffxml_filename)